In [35]:
from PIL import Image
from time import time_ns
import math
import numpy as np

In [36]:
def getImages():
  img_first = Image.open('input/first.png', 'r').convert('RGB')
  img_second = Image.open('input/second.png', 'r').convert('RGB')
  img_third = Image.open('input/third.png', 'r').convert('RGB')
  return {'first': img_first, 'second': img_second, 'third': img_third}

In [37]:
def printAudit(test_name, delta_time, audit):
  print(f'\nTest {test_name} passed: {(delta_time) / math.pow(10, 9)}s')
  for i in audit:
    name = i.get('name')
    input_size = i.get('input_size')
    output_size = i.get('output_size')
    print(f'Image {name}; input_size: {input_size}; output_size: {output_size}')

In [38]:
def grayscale(image):
  result = Image.new('L', (image.width, image.height))

  for x in range(result.width):  #цвет в полутон
    for y in range(result.height):
      pixel = image.getpixel((x, y))
      new_pixel = int(round(pixel[0] * 0.3 + pixel[1] * 0.59 + pixel[2] * 0.11))
      result.putpixel((x, y), new_pixel)

  return result

operator_prewitt_x = np.array([[-1, -1, -1], 
                               [ 0,  0,  0],
                               [ 1,  1,  1]])
operator_prewitt_y = np.array([[-1, 0, 1], 
                               [-1, 0, 1],
                               [-1, 0, 1]])

In [39]:
def contour_selection(image, operator_x, operator_y, window_size):
  width, height = image.size
  S = 2.5   #порог бинаризации
  
  gray_arr = np.asarray(image, dtype=np.uint8)
  
  G_x_matrix = np.zeros(shape=(height, width))
  G_y_matrix = np.zeros(shape=(height, width))
  G_matrix = np.zeros(shape=(height, width))
  
  padded_gray_arr = np.pad(gray_arr, ((1, 1), (1, 1)), mode='constant')  #однопиксельные отступы вокруг, чтобы градиент на краях был в норме
  for i in range(1, height+1):    #перебор пикселей
    for j in range(1, width+1): 
      i_min = max(i - window_size // 2, 0)
      i_max = min(i + window_size // 2, height+1)
      j_min = max(j - window_size // 2, 0)
      j_max = min(j + window_size // 2, width+1)
      
      window_matrix = padded_gray_arr[i_min:i_max+1, j_min:j_max+1] #матрица значений для окна свертки
      
      G_x = np.sum(operator_x * window_matrix) #компоненты градиента
      G_y = np.sum(operator_y * window_matrix)
      G = np.abs(G_x) + np.abs(G_y)

      G_x_matrix[i-1, j-1] = int(round(G_x))
      G_y_matrix[i-1, j-1] = int(round(G_y))
      G_matrix[i-1, j-1] = int(round(G))
        
  G_x_matrix_normalized = np.abs(G_x_matrix / np.max(G_x_matrix) * 255) #не выходить за ограду от 0 до 255!!
  G_y_matrix_normalized = np.abs(G_y_matrix / np.max(G_y_matrix) * 255)
  G_matrix_normalized = G_matrix / np.max(G_matrix) * 255
  
  t = np.mean(G_matrix_normalized) * S   #порог бинаризации(среднее на S)

  gXImg = Image.fromarray(G_x_matrix_normalized.astype(np.uint8), mode='L')
  gYImg = Image.fromarray(G_y_matrix_normalized.astype(np.uint8), mode='L')

  g_img = Image.fromarray(G_matrix_normalized.astype(np.uint8), mode='L')
  binary_img = Image.fromarray((G_matrix_normalized > t).astype(np.uint8) * 255, mode = 'L') # порог бинаризации к суммарной матрице, выше - снежок, ниже - уголек

  return gXImg, gYImg, g_img, binary_img

In [40]:
def contour_selection_test(operator_x, operator_y, window_size: int):
  images = getImages()

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    grayscale_image = grayscale(image)
    g_x_img, g_y_img, g_img, binary_img = contour_selection(grayscale_image, operator_x, operator_y, window_size)

    output_name_x = f'output/{image_name}_x.processed.png'
    output_name_y = f'output/{image_name}_y.processed.png'
    output_name_binary = f'output/{image_name}_binary.processed.png'
    output_name = f'output/{image_name}.processed.png'
    output_name_grayscale = f'output/{image_name}_grayscale.processed.png'

    audit.append({
      'name': output_name_x,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{g_x_img.size[0]}x{g_x_img.size[1]}'
    })
    audit.append({
      'name': output_name_y,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{g_y_img.size[0]}x{g_y_img.size[1]}'
    })
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{g_img.size[0]}x{g_img.size[1]}'
    })
    audit.append({
      'name': output_name_binary,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{binary_img.size[0]}x{binary_img.size[1]}'
    })

    g_x_img.save(output_name_x)
    g_y_img.save(output_name_y)
    g_img.save(output_name)
    binary_img.save(output_name_binary)
    grayscale_image.save(output_name_grayscale)

  end_time = time_ns()
  printAudit('contour_selection', end_time - start_time, audit)

In [41]:
def test():
  contour_selection_test(operator_prewitt_x, operator_prewitt_y, 3)

In [42]:
if __name__ == '__main__':
  test()


Test contour_selection passed: 387.3407423s
Image output/first_x.processed.png; input_size: 3639x2381; output_size: 3639x2381
Image output/first_y.processed.png; input_size: 3639x2381; output_size: 3639x2381
Image output/first.processed.png; input_size: 3639x2381; output_size: 3639x2381
Image output/first_binary.processed.png; input_size: 3639x2381; output_size: 3639x2381
Image output/second_x.processed.png; input_size: 1280x813; output_size: 1280x813
Image output/second_y.processed.png; input_size: 1280x813; output_size: 1280x813
Image output/second.processed.png; input_size: 1280x813; output_size: 1280x813
Image output/second_binary.processed.png; input_size: 1280x813; output_size: 1280x813
Image output/third_x.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/third_y.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/third.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/third_binary.processed.png; input_size: 